In [ ]:
import numpy as np
import pymongo
import gridfs
import pickle
from pymongo import MongoClient
from lz4.frame import compress,decompress
import tensorlayer as  tl


class MnistDB:
    def __init__(
        self,
        ip = 'localhost',
        port = 27017,
        db_name = 'db_name',
        user_name = None,
        password = 'password',
    ):
        ## connect mongodb
        client = MongoClient(ip, port)
        self.db = client[db_name]
        if user_name != None:
            self.db.authenticate(user_name, password)
                
        ## define file system (Buckets)
        self.datafs = gridfs.GridFS(self.db, collection="datafs")
    
    def import_data(self,X,y,args={}):

        t=time.time()
        s=pickle.dumps(X,protocol=2)
        t_1=time.time()-t
        s1=compress(s,compression_level=3)
        t_2=time.time()-t
        iid=self.datafs.put(s1,filename="imageData")
        t2=time.time()
        print len(s)
        print len(s1)

        print "insert time"
        print t_1
        print t_2
        print t2-t
        p=[]
        for i in range(X.shape[0]):

            ip={"id":i,'imageData':iid,'label':np.asscalar(y[i])}
            ip.update(args)
            p.append(ip)

        t1end=time.time()-t2
        print "memory sorting time"
        print t1end
        t3=time.time()
        rl=self.db.DataSet.insert_many(p)
        t2end=time.time()-t3

        print "insert many time"
        print t2end
        
    def find_data(self,args={'type':"train",'id':{"$lt":500}}):

        t=time.time()
        pc=self.db.DataSet.find(args)
        flist=pc.distinct('imageData')
        t1=time.time()-t
        print t1
        fldict={}
        for f in flist:
            t=time.time()
            s=self.datafs.get(f).read()
            s2=decompress(s)
            t2=time.time()-t
            print "reading time"
            print t2
            t3=time.time()
            fldict[f]=pickle.loads(s2)
            t4=time.time()
            print "des time "
            print t4-t3

            print "TotalTime"
            print t4-t


        print pc.count()

        t2=time.time()-t
        print t2


        t5=time.time()
        rt=[(fldict[x['imageData']][x['id']].reshape(1,784),x['label']) for x in pc]
        t6=time.time()-t5
        dl=zip(*rt)
        d=np.concatenate(dl[0])
        l=np.array(dl[1])

        
        print "training reading"
        print t6
      
        return d,l
    
    def generator_data(self,batch_size=20,args={'type':"train",'id':{"$lt":500}}):
            
        pc=self.db.DataSet.find(args)
        flist=pc.distinct('imageData')
        fldict={}
        for f in flist:
            s=self.datafs.get(f).read()
            s2=decompress(s)
            fldict[f]=pickle.loads(s2)
            
        print pc.count()

        for i in range(0,pc.count(),batch_size):
          
            pc.rewind()
          
         
            rt=[(fldict[x['imageData']][x['id']].reshape(1,784),x['label']) for x in pc[i:i+batch_size]]
      
            dl=zip(*rt)
            d=np.concatenate(dl[0])
            l=np.array(dl[1])
        
            yield d,l

        
        
        
 


X_train, y_train, X_val, y_val, X_test, y_test = tl.files.load_mnist_dataset(shape=(-1,784))

X_train = np.asarray(X_train, dtype=np.float32)
y_train = np.asarray(y_train, dtype=np.int32)
X_val = np.asarray(X_val, dtype=np.float32)
y_val = np.asarray(y_val, dtype=np.int32)
X_test = np.asarray(X_test, dtype=np.float32)
y_test = np.asarray(y_test, dtype=np.int32)

print('X_train.shape', X_train.shape)
print('y_train.shape', y_train.shape)
print('X_val.shape', X_val.shape)
print('y_val.shape', y_val.shape)
print('X_test.shape', X_test.shape)
print('y_test.shape', y_test.shape)
print('X %s   y %s' % (X_test.dtype, y_test.dtype))

mb=MnistDB()


d,l=mb.find_data()

g=mb.generator_data(batch_size=100,args={'type':"train",'id':{"$lt":500}})


In [ ]:
#! /usr/bin/python
# -*- coding: utf8 -*-
import tensorflow as tf
import tensorlayer as tl
import numpy as np
import time
import math


import uuid

import pymongo
import gridfs
import pickle
from pymongo import MongoClient
from datetime import datetime

import inspect

def AutoFill(func):
    def func_wrapper(self,*args,**kwargs):
        d=inspect.getcallargs(func,self,*args,**kwargs)
        d['args'].update({"studyID":self.studyID})
        return  func(**d)             
    return func_wrapper



        


class TensorDB(object):
    """TensorDB is a MongoDB based manager that help you to manage data, model and logging.

    Parameters
    -------------
    ip : string, localhost or IP address.
    port : int, port number.
    db_name : string, database name.
    user_name : string, set to None if it donnot need authentication.
    password : string.

    Properties
    ------------
    db : ``pymongo.MongoClient[db_name]``, xxxxxx
    datafs : ``gridfs.GridFS(self.db, collection="datafs")``, xxxxxxxxxx
    modelfs : ``gridfs.GridFS(self.db, collection="modelfs")``,
    paramsfs : ``gridfs.GridFS(self.db, collection="paramsfs")``,
    db.Params : Collection for
    db.TrainLog : Collection for
    db.ValidLog : Collection for
    db.TestLog : Collection for

    Dependencies
    -------------
    1 : MongoDB, as TensorDB is based on MongoDB, you need to install it in your
       local machine or remote machine.
    2 : pip install pymongo, for MongoDB python API.

    Optional Tools
    ----------------
    1 : You may like to install MongoChef or Mongo Management Studo APP for
       visualizing or testing your MongoDB.
    """
    def __init__(
        self,
        ip = 'localhost',
        port = 27017,
        db_name = 'db_name',
        user_name = None,
        password = 'password',
        studyID=None
    ):
        ## connect mongodb
        client = MongoClient(ip, port)
        self.db = client[db_name]
        if user_name != None:
            self.db.authenticate(user_name, password)
            
        
        if studyID is None:
            self.studyID=str(uuid.uuid1())
        else:
            self.studyID=studyID
            
        ## define file system (Buckets)
        self.datafs = gridfs.GridFS(self.db, collection="datafs")
        self.modelfs = gridfs.GridFS(self.db, collection="modelfs")
        self.paramsfs = gridfs.GridFS(self.db, collection="paramsfs")
        self.archfs=gridfs.GridFS(self.db,collection="ModelArchitecture")
        ##
        print("[TensorDB] Connect SUCCESS {}:{} {} {}".format(ip, port, db_name, user_name))

        self.ip = ip
        self.port = port
        self.db_name = db_name
        self.user_name = user_name

    # def save_bulk_data(self, data=None, filename='filename'):
    #     """ Put bulk data into TensorDB.datafs, return file ID.
    #     When you have a very large data, you may like to save it into GridFS Buckets
    #     instead of Collections, then when you want to load it, XXXX
    #
    #     Parameters
    #     -----------
    #     data : serialized data.
    #     filename : string, GridFS Buckets.
    #
    #     References
    #     -----------
    #     - MongoDB find, xxxxx
    #     """
    #     s = time.time()
    #     f_id = self.datafs.put(data, filename=filename)
    #     print("[TensorDB] save_bulk_data: {} took: {}s".format(filename, round(time.time()-s, 2)))
    #     return f_id
    #
    # def save_collection(self, data=None, collect_name='collect_name'):
    #     """ Insert data into MongoDB Collections, return xx.
    #
    #     Parameters
    #     -----------
    #     data : serialized data.
    #     collect_name : string, MongoDB collection name.
    #
    #     References
    #     -----------
    #     - MongoDB find, xxxxx
    #     """
    #     s = time.time()
    #     rl = self.db[collect_name].insert_many(data)
    #     print("[TensorDB] save_collection: {} took: {}s".format(collect_name, round(time.time()-s, 2)))
    #     return rl
    #
    # def find(self, args={}, collect_name='collect_name'):
    #     """ Find data from MongoDB Collections.
    #
    #     Parameters
    #     -----------
    #     args : dictionary, arguments for finding.
    #     collect_name : string, MongoDB collection name.
    #
    #     References
    #     -----------
    #     - MongoDB find, xxxxx
    #     """
    #     s = time.time()
    #
    #     pc = self.db[collect_name].find(args)  # pymongo.cursor.Cursor object
    #     flist = pc.distinct('f_id')
    #     fldict = {}
    #     for f in flist: # you may have multiple Buckets files
    #         # fldict[f] = pickle.loads(self.datafs.get(f).read())
    #         # s2 = time.time()
    #         tmp = self.datafs.get(f).read()
    #         # print(time.time()-s2)
    #         fldict[f] = pickle.loads(tmp)
    #         # print(time.time()-s2)
    #         # exit()
    #     # print(round(time.time()-s, 2))
    #     data = [fldict[x['f_id']][x['id']] for x in pc]
    #     data = np.asarray(data)
    #     print("[TensorDB] find: {} get: {} took: {}s".format(collect_name, pc.count(), round(time.time()-s, 2)))
    #     return data

    # def del_data(self, data, args={}):
    #     pass
    #
    # def save_model(self):
    #     pass
    #
    # def load_model(self):
    #     pass
    #
    # def del_model(self):
    #     pass

    def __autofill(self,args):
        return args.update({'studyID':self.studyID})
    
    def __serialization(self,ps):
        return pickle.dumps(ps, protocol=2)
    
    def __deserialization(self,ps):
        return pickle.loads(ps)
    
    def save_params(self, params=[], args={}):#, file_name='parameters'):
        """ Save parameters into MongoDB Buckets, and save the file ID into Params Collections.

        Parameters
        ----------
        params : a list of parameters
        args : dictionary, item meta data.

        Returns
        ---------
        f_id : the Buckets ID of the parameters.
        """
        
        self.__autofill(args)
        s = time.time()
        f_id = self.paramsfs.put(self.__serialization(params))#, file_name=file_name)
        args.update({'f_id': f_id, 'time': datetime.utcnow()})
        self.db.Params.insert_one(args)
        # print("[TensorDB] Save params: {} SUCCESS, took: {}s".format(file_name, round(time.time()-s, 2)))
        print("[TensorDB] Save params: SUCCESS, took: {}s".format(round(time.time()-s, 2)))
        return f_id
    
    @AutoFill
    def find_one_params(self, args={}):
        """ Find one parameter from MongoDB Buckets.

        Parameters
        ----------
        args : dictionary, find items.

        Returns
        --------
        params : the parameters, return False if nothing found.
        f_id : the Buckets ID of the parameters, return False if nothing found.
        """
       
        s = time.time()
        d = self.db.Params.find_one(args)

        if d is not None:
            f_id = d['f_id']
        else:
            print("[TensorDB] FAIL! Cannot find: {}".format(args))
            return False, False
        try:
            params = self.__deserialization(self.paramsfs.get(f_id).read())
            print("[TensorDB] Find one params SUCCESS, {} took: {}s".format(args, round(time.time()-s, 2)))
            return params, f_id
        except:
            return False, False
        
    @AutoFill
    def find_all_params(self, args={}):
        """ Find all parameter from MongoDB Buckets

        Parameters
        ----------
        args : dictionary, find items

        Returns
        --------
        params : the parameters, return False if nothing found.
        
        """
        
        s = time.time()
        pc = self.db.Params.find(args)

        if pc is not None:
            f_id_list = pc.distinct('f_id')
            params = []
            for f_id in f_id_list: # you may have multiple Buckets files
                tmp = self.paramsfs.get(f_id).read()
                params.append(self.__deserialization(tmp))
        else:
            print("[TensorDB] FAIL! Cannot find any: {}".format(args))
            return False

        print("[TensorDB] Find all params SUCCESS, took: {}s".format(round(time.time()-s, 2)))
        return params
    
    @AutoFill
    def del_params(self, args={}):
        """ Delete params in MongoDB uckets.

        Parameters
        -----------
        args : dictionary, find items to delete, leave it empty to delete all parameters.
        """
    
        pc = self.db.Params.find(args)
        f_id_list = pc.distinct('f_id')
        # remove from Buckets
        for f in f_id_list:
            self.paramsfs.delete(f)
        # remove from Collections
        self.db.Params.remove(args)

        print("[TensorDB] Delete params SUCCESS: {}".format(args))
    
 
    def _print_dict(self, args):
        # return " / ".join(str(key) + ": "+ str(value) for key, value in args.items())
        
        string = ''
        for key, value in args.items():
            if key is not '_id':
                string += str(key) + ": "+ str(value) + " / "
        return string
    
    @AutoFill
    def save_job(self, script=None, args={}):
        """Save the job.

        Parameters
        -----------
        script : a script file name or None.
        args : dictionary, items to save.

        Examples
        ---------
        >>> # Save your job
        >>> db.save_job('your_script.py', {'job_id': 1, 'learning_rate': 0.01, 'n_units': 100})
        >>> # Run your job
        >>> temp = db.find_one_job(args={'job_id': 1})
        >>> print(temp['learning_rate'])
        ... 0.01
        >>> import _your_script
        ... running your script
        """
        self.__autofill(args)
        if script is not None:
            _script = open(script, 'rb').read()
            args.update({'script': _script, 'script_name': script})
        # _result = self.db.Job.insert_one(args)
        _result = self.db.Job.replace_one(args, args, upsert=True)
        _log = self._print_dict(args)
        print("[TensorDB] Save Job: script={}, args={}".format(script, args))
        return _result

    @AutoFill
    def find_one_job(self, args={}):
        """ Find one job from MongoDB Job Collections.

        Parameters
        ----------
        args : dictionary, find items.

        Returns
        --------
        dictionary : contains all meta data and script.
        """
        
     
        temp = self.db.Job.find_one(args)

        if temp is not None:
            if 'script_name' in temp.keys():
                f = open('_' + temp['script_name'], 'wb')
                f.write(temp['script'])
                f.close()
            print("[TensorDB] Find Job: {}".format(args))
        else:
            print("[TensorDB] FAIL! Cannot find any: {}".format(args))
            return False
        
        return temp
    
    @AutoFill
    def train_log(self, args={}):
        """Save the training log.

        Parameters
        -----------
        args : dictionary, items to save.

        Examples
        ---------
        >>> db.train_log(time=time.time(), {'loss': loss, 'acc': acc})
        """
     
        _result = self.db.TrainLog.insert_one(args)
        _log = self._print_dict(args)
        #print("[TensorDB] TrainLog: " +_log)
        return _result
    
    @AutoFill
    def del_train_log(self, args={}):
        """ Delete train log.

        Parameters
        -----------
        args : dictionary, find items to delete, leave it empty to delete all log.
        """
     
        self.db.TrainLog.delete_many(args)
        print("[TensorDB] Delete TrainLog SUCCESS")
    
    @AutoFill
    def valid_log(self, args={}):
        """Save the validating log.

        Parameters
        -----------
        args : dictionary, items to save.

        Examples
        ---------
        >>> db.valid_log(time=time.time(), {'loss': loss, 'acc': acc})
        """
        
        _result = self.db.ValidLog.insert_one(args)
        # _log = "".join(str(key) + ": " + str(value) for key, value in args.items())
        _log = self._print_dict(args)
        print("[TensorDB] ValidLog: " +_log)
        return _result
    
    @AutoFill
    def del_valid_log(self, args={}):
        """ Delete validation log.

        Parameters
        -----------
        args : dictionary, find items to delete, leave it empty to delete all log.
        """
        self.db.ValidLog.delete_many(args)
        print("[TensorDB] Delete ValidLog SUCCESS")
    
    @AutoFill
    def test_log(self, args={}):
        """Save the testing log.

        Parameters
        -----------
        args : dictionary, items to save.

        Examples
        ---------
        >>> db.test_log(time=time.time(), {'loss': loss, 'acc': acc})
        """
      
        _result = self.db.TestLog.insert_one(args)
        # _log = "".join(str(key) + str(value) for key, value in args.items())
        _log = self._print_dict(args)
        print("[TensorDB] TestLog: " +_log)
        return _result
    
    @AutoFill
    def del_test_log(self, args={}):
        """ Delete test log.

        Parameters
        -----------
        args : dictionary, find items to delete, leave it empty to delete all log.
        """

        self.db.TestLog.delete_many(args)
        print("[TensorDB] Delete TestLog SUCCESS")

    def __str__(self):
        _s = "[TensorDB] Info:\n"
        _t = _s + "    " + str(self.db)
        return _t
        
    @AutoFill
    def save_model_architecture(self,s,args={}):
        self.__autofill(args)
        fid=self.archfs.put(s,filename="modelarchitecture")
        args.update({"fid":fid})
        self.db.march.insert_one(args)
        
    @AutoFill 
    def load_model_architecture(self,args={}):
     
        d = self.db.march.find_one(args)
        if d is not None:
            fid = d['fid']
            print (d)
            print (fid)
            "print find"
        else:
            print("[TensorDB] FAIL! Cannot find: {}".format(args))
            print ("no idtem")
            return False, False
        try:
            archs = self.archfs.get(fid).read()
            '''print("[TensorDB] Find one params SUCCESS, {} took: {}s".format(args, round(time.time()-s, 2)))'''
            return archs, fid
        except Exception as e:
            print ("exception")
            print (e)
            return False, False
    
    
    def push_job(self,margs, wargs,dargs,epoch):
        
        ms,mid=self.load_model_architecture(margs)
        weight,wid=self.find_one_params(wargs)
        args={"weight":wid,"model":mid,"dargs":dargs,"epoch":epoch,"time":datetime.utcnow(),"Running":False}
        self.__autofill(args)
        self.db.JOBS.insert_one(args)
    
    def peek_job(self):
        args={'Running':False}
        self.__autofill(args)
        m=self.db.JOBS.find_one(args)
        print(m)
        if m is None:
            return False
        
        s=self.paramsfs.get(m['weight']).read()
        w=self.__deserialization(s)
        
        ach=self.archfs.get(m['model']).read()
        
        return m['_id'], ach,w,m["dargs"],m['epoch']
    
    def run_job(self,jid):
        self.db.JOBS.find_one_and_update({'_id':jid},{'$set': {'Running': True,"Since":datetime.utcnow()}})
 

        
    def del_job(self,jid):
        self.db.JOBS.find_one_and_update({'_id':jid},{'$set': {'Running': True,"Finished":datetime.utcnow()}})
        
    
db = TensorDB(ip='localhost', port=27017, db_name='mnist', user_name=None, password=None,studyID='mytest')

In [ ]:

class DBLogger:
    
    def __init__(self,db,model):
        self.db=db
        self.model=model
        
    def on_train_begin(self,logs={}):
        print "start"
    
    def on_train_end(self,logs={}):
        print "end"
    
    def on_epoch_begin(self,epoch,logs={}):
        self.epoch=epoch
        return
    
    def on_epoch_end(self, epoch, logs={}):
        print("ending")
        print(epoch)
        logs['epoch']=epoch
        logs['time']=datetime.utcnow()
        print logs
        db.valid_log(logs)
        w=self.model.Params
        db.save_params(w,logs)
        
    def on_batch_begin(self, batch,logs={}):
        self.t=time.time()
        self.losses = []

    def on_batch_end(self, batch, logs={}):
        self.t2=time.time()-self.t
        logs['acc']=np.asscalar(logs['acc'])
        #logs['loss']=np.asscalar(logs['loss'])
        logs['step_time']=self.t2
        logs['time']=datetime.utcnow()
        logs['epoch']=self.epoch
        self.db.train_log(logs)
        
    

In [ ]:



code='''


class Model:
    def __init__(self, name="tensorDB", reuse=False,args={}):
        self.sess=tf.InteractiveSession()
        x = tf.placeholder(tf.float32, shape=[None, 784], name='x')
        y_ = tf.placeholder(tf.int32, shape=[None, ], name='y_')

        with tf.variable_scope(name, reuse=reuse):
            tl.layers.set_name_reuse(reuse)
      
        
            network = tl.layers.InputLayer(x, name='input_layer')
            network = tl.layers.DropoutLayer(network, keep=0.8, name='drop1')
            network = tl.layers.DenseLayer(network, n_units=800,
                                            act = tf.nn.relu, name='relu1')
            network = tl.layers.DropoutLayer(network, keep=0.5, name='drop2')
            network = tl.layers.DenseLayer(network, n_units=800,
                                            act = tf.nn.relu, name='relu2')
            network = tl.layers.DropoutLayer(network, keep=0.5, name='drop3')
            network = tl.layers.DenseLayer(network, n_units=10,
                                            act = tf.identity,
                                            name='output_layer')
        y = network.outputs
        y_op = tf.argmax(tf.nn.softmax(y), 1)
        # cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(y, y_))
        cost = tl.cost.cross_entropy(y, y_, name='cost')
        
        
        learning_rate = 0.0001
 
        train_op = tf.train.AdamOptimizer(learning_rate, beta1=0.9, beta2=0.999,epsilon=1e-08, use_locking=False).minimize(cost)
        
        
        tl.layers.initialize_global_variables(self.sess)
        
        
        network.print_params()
        network.print_layers()
        
        self.x=x
        self.y_=y_
        self.network=network
        self.params=network.all_params
        self.predict=y_op
        
        self.loss=cost
        self.train_op=train_op

    def __del__(self):
        self.sess.close()
   
        
    @property    
    def Predictor(self):
        return self.y_op
        
    
    @property
    def Optimisor(self):
        return self.train_op

    @property
    def Accuracy(self):
        return self.loss
    
    @property
    def Params(self):
        return self.sess.run(self.params)
       

    @Params.setter
    def Params(self,x):
        tl.files.assign_params( self.sess,x,self.network)
    
    @property
    def Input(self):
        return self.x
    
    
    
        
    def fit(self,n_epoch, X_train, y_train,batch_size, callback=[]):
        
        for c in callback:
            c.on_train_begin()
            
        
        for epoch in range(n_epoch):
            
            for c in callback:
                c.on_epoch_begin(epoch,{})
            w=0
            
            for X_train_a, y_train_a in tl.iterate.minibatches(X_train, y_train,batch_size, shuffle=True):
            
                for c in callback:
                    c.on_batch_begin(w)
                    
                feed_dict = {self.x: X_train_a, self.y_: y_train_a}
            
            
                feed_dict.update( self.network.all_drop )    # enable dropout or dropconnect layers
                [dm,dc]=self.sess.run([self.train_op,self.loss], feed_dict=feed_dict)
            
                for c in callback:
                    c.on_batch_end(w,{'acc':dc,'size':X_train_a.shape[0]})
            
            
            for c in callback:
                c.on_epoch_end(epoch,{})
            w=0

               
            
        for c in callback:
            c.on_train_end()
     
    def fit_generator(self, generator, callback=[]):
        for c in callback:
            c.on_train_begin()
            
        
        for epoch in range(1):
            
            for c in callback:
                c.on_epoch_begin(1,{})
            w=0
            
            for X_train_a, y_train_a in generator:
                
                print "step"
            
                for c in callback:
                    c.on_batch_begin(w)
                    
                feed_dict = {self.x: X_train_a, self.y_: y_train_a}
            
            
                feed_dict.update( self.network.all_drop )    # enable dropout or dropconnect layers
                [dm,dc]=self.sess.run([self.train_op,self.loss], feed_dict=feed_dict)
            
                for c in callback:
                    c.on_batch_end(w,{'acc':dc,'size':X_train_a.shape[0]})
            
            
            for c in callback:
                c.on_epoch_end(1,{})
            w=0

               
            
        for c in callback:
            c.on_train_end()
     
        
'''
        
    
            



db.save_model_architecture(code,{'name':"mlp"})



exec code
m=Model(name="mytest2",reuse=False)
m.fit(10,X_train,y_train,100,[DBLogger(db,m)])




In [ ]:
#job
for i in range(100):
    db.push_job({"name":"mlp"},{"epoch":9},{"type":"train"},10)
    db.push_job({"name":"mlp"},{"epoch":9},{"type":"train"},10)
    db.push_job({"name":"mlp"},{"epoch":9},{"type":"val"},10)
    db.push_job({"name":"mlp"},{"epoch":9},{"type":"val"},10)
    db.push_job({"name":"mlp"},{"epoch":9},{"type":"val"},10)
    db.push_job({"name":"mlp"},{"epoch":9},{"type":"val"},10)
    

In [ ]:
import tensorflow as tf
import tensorlayer as tl
import time
import uuid


def run_job(db,mb):
    if db.peek_job():
        j, m, w, darg, ep=db.peek_job()

        print ep
        print type(darg)

        db.run_job(j)
        exec m
        mm=Model(name=str(uuid.uuid1()),reuse=False)
        mm.Params=w;
        x,y=mb.find_data(darg)
        mm.fit(ep,x,y,100,[DBLogger(db,mm)])
        del mm
        db.del_job(j)
        return True
    
    return False

while run_job(db,mb):
    print "new job is done"